In [52]:
import os
import pandas as pd
import numpy as np
from mol_lib import xtb_isomers, FromXtbToSpinM

with open("/Users/tinaccil/Documents/GitHub/woon_query/data_analysis/dataframe/molecules_info.csv","r") as data:
	df_mol = pd.read_csv(data, delimiter = '\t')
with open("/Users/tinaccil/Documents/GitHub/woon_query/data_analysis/dataframe/network.csv","r") as data:
	df_net = pd.read_csv(data, delimiter = '\t')

#with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#    display(df_mol)

,name,formula,ID,Na,Mass,state_woon,multiplicity_woon,energy(kJ/mol)_woon,spin_input_xtb,multiplicity_xtb,energy(kJ/mol)_xtb,pwd_xyz_woon,pwd_xyz_xtb
0,H,H1,H1_0,1,1.00784,2Sigma,2,-1312.252021,0.5,2,-1033.089679,/Users/tinaccil/Documents/GitHub/woon_query/xy...,/Users/tinaccil/Documents/GitHub/molecules_dat...
1,C,C1,C1_0,1,12.01070,3Pi,3,-99193.456115,0.0,3,-4713.065777,/Users/tinaccil/Documents/GitHub/woon_query/xy...,/Users/tinaccil/Documents/GitHub/molecules_dat...
2,N,N1,N1_0,1,14.00670,4Sigma,4,-143128.457716,0.5,4,-6851.121943,/Users/tinaccil/Documents/GitHub/woon_query/xy...,/Users/tinaccil/Documents/GitHub/molecules_dat...
3,O,O1,O1_0,1,15.99900,3Sigma,3,-196844.267180,0.0,3,-9896.621624,/Users/tinaccil/Documents/GitHub/woon_query/xy...,/Users/tinaccil/Documents/GitHub/molecules_dat...
4,H2,H2,H2_0,2,2.01568,1Sigma,1,-3078.757850,0.0,1,-2580.042454,/Users/tinaccil/Documents/GitHub/woon_query/xy...,/Users/tinaccil/Documents/GitHub/molecules_dat...
5,CH,H1C1,H1C1_0,2,13.01854,2Pi,2,-100852.502783,0.5,2,-6251.102486,/Users/tinaccil/Documents/GitHub/woon_query/xy...,/Users/tinaccil/Documents/GitHub/molecules_dat...
6,NH,H1N1,H1N1_0,2,15.01454,3Sigma,3,-144782.699717,0.0,3,-8413.109096,/Users/tinaccil/Documents/GitHub/woon_query/xy...,/Users/tinaccil/Documents/GitHub/molecules_dat...
7,OH,H1O1,H1O1_0,2,17.00684,2Pi,2,-198607.123562,0.5,2,-11626.705754,/Users/tinaccil/Documents/GitHub/woon_query/xy...,/Users/tinaccil/Documents/GitHub/molecules_dat...
8,C2,C2,C2_0,2,24.02140,1Sigma,1,-198977.269422,0.0,1,-10621.964202,/Users/tinaccil/Documents/GitHub/woon_query/xy...,/Users/tinaccil/Documents/GitHub/molecules_dat...
9,CN,C1N1,C1N1_0,2,26.01740,2Sigma,2,-243047.381968,0.5,2,-12854.387667,/Users/tinaccil/Documents/GitHub/woon_query/xy...,/Users/tinaccil/Documents/GitHub/molecules_dat...


In [ ]:
'''
tmp_w = df_mol['multiplicity_woon'].tolist()
tmp_x = df_mol['multiplicity_xtb'].tolist()

tmp_b = []
for i,item in enumerate(tmp_w):
	if (int(tmp_w[i]) - int(tmp_x[i])) == 0:
		tmp_b.append(True)
	else:
		tmp_b.append(False)

df_mol['bool'] = tmp_b
df_mol_f = df_mol[df_mol['bool'] == False]
df_mol = df_mol.drop(["bool"], axis=1)
'''